### Accuracy Impacts of Augmentation on Testsets

This notebook uses the models the team trained on augmented datasets to determine what impact evaluating testsets that have also been augmented has on model accuracy.  While the team's focus is on shrinking the distribution gap observed between the CIFAR 10 and 10.1 testsets, the results are examined from two dimensions:
  1. What, if any, effect is observed on the distribution gap when comparing model evaluations on an augemented CIFAR 10 testset to an augmented CIFAR 10.1 tesetset.
  2. What, if any, effect is observed in model accuracy when comparing the model performs predicting classes for the unaugmented CIFAR X testset versus the augmented CIFAR X testset?

In [1]:
import boto3
from collections import defaultdict
import glob
import os
import pandas as pd
import re

pd.set_option('display.max_rows', 500)

In [9]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 3.0 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 779 kB 26.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 49.3 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 57.3 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 57.5 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 46.9 MB/s eta 0:00:01
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=9e7af51dbe45d92ece579d1cdeac7f5c93a8d6ecea4017f30f70f0ecf2802027
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679
Successfully built absl-py
You should consider upgr

In [2]:
# S3 Bucket where the model checkpoints exist (Set based on AWS instance used)
# bucket = 'sagemaker-june29'
bucket = 'sagemaker-may29'

# Directory path within S3 to the model results
prefix = 'sagemaker/results/original-models/'

# Local destination directory 
destdir = '/home/ec2-user/SageMaker/checkpoints/'

# File path to model pytorch files
pytorch_models = '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/'

# Dictionary to look up matching config files with
config_files = defaultdict(lambda: "default", key="skip")

config_files['wrn_28_10'] = "wrn.yaml"
config_files['resnet_basic_32'] = "resnet.yaml"
config_files['densenet_BC_100_12'] = "densenet.yaml"
config_files['resnext_29_4x64d'] =  "resnext.yaml"

In [3]:
# Retrieve a list the checkpoint files 
def get_chpt_files(bucket, prefix):

    s3 = boto3.client('s3')
    refined_checkpoints = []
    basetraining_checkpoints = []
    kwargs = {'Bucket': bucket, 'Prefix': prefix}

    resp = s3.list_objects_v2(**kwargs)
    for obj in resp['Contents']:
        key = obj['Key']
        if key.endswith("00400.pth"):
            basetraining_checkpoints.append(key)
        if key.endswith("00050.pth"):
            refined_checkpoints.append(key)
            
    return basetraining_checkpoints, refined_checkpoints



# Download a checkpoint file and write it to the destination directory
def download_chpt(bucket, destdir, file):
        
    s3 = boto3.resource('s3')
    filesplit = file.split("/")
    filename = str(filesplit[-1])
    model = str(filesplit[3])
    destination = destdir + model + '_' + filename
    s3.Bucket(bucket).download_file(file, destination)
        

In [6]:
# Create the destination directory if it doesn't exist
if not os.path.exists(destdir):
        os.mkdir(destdir)

# Download the model checkpoints
basetraining_checkpoints, refined_checkpoints = get_chpt_files(bucket, prefix)

for cp in basetraining_checkpoints + refined_checkpoints:
    download_chpt(bucket, destdir, cp)

In [4]:
os.chdir(destdir)
listing = glob.glob(destdir + "*pth")

RA_2_5_Datasets = ['CIFAR10_RA_2_5', 'CIFAR101_RA_2_5']
RA_1_20_Datasets = ['CIFAR10_RA_1_20', 'CIFAR101_RA_1_20']
RA_2_20_Datasets = ['CIFAR10_RA_2_20', 'CIFAR101_RA_2_20']
RA_3_20_Datasets = ['CIFAR10_RA_3_20', 'CIFAR101_RA_3_20']

In [5]:
result = None

def evalute_models(datasets, checkpoints):
    for ds in datasets:
        for chpt in checkpoints:
            
            print(chpt)

            filesplit = chpt.split("/")
            filename = str(filesplit[-1])
            model = filename.split("_")[0]

            os.chdir(pytorch_models)
            results_folder = '/home/ec2-user/SageMaker/experiments/augmented_testsets/' +  filename

            if 'wrn_28_10' in filename:
                print("Test dataset:", ds)
                !python evaluate.py --config configs/cifar/wrn.yaml \
                   model.wrn.depth 28 \
                   model.wrn.widening_factor 10 \
                   dataset.name $ds \
                   test.checkpoint $chpt \
                   test.output_dir $results_folder

            elif 'resnet_basic_32' in filename: 
                print("Test dataset:", ds)
                !python evaluate.py --config configs/cifar/resnet.yaml \
                   model.resnet.depth 32 \
                   test.batch_size 128 \
                   dataset.name $ds \
                   test.checkpoint $chpt \
                   test.output_dir $results_folder

            elif 'densenet_BC_100_12' in filename:
                print("Test dataset:", ds)
                !python evaluate.py --config configs/cifar/densenet.yaml \
                   model.densenet.depth 100 \
                   model.densenet.growth_rate 12 \
                   test.batch_size 64 \
                   dataset.name $ds \
                   test.checkpoint $chpt \
                   test.output_dir $results_folder

            elif 'resnext_29_4x64d' in filename:
                print("Test dataset:", ds)
                !python evaluate.py --config configs/cifar/resnext.yaml \
                   model.resnext.cardinality 4 \
                   test.batch_size 128 \
                   dataset.name $ds \
                   test.checkpoint $chpt \
                   test.output_dir $results_folder


In [18]:
%%capture cap --no-stderr

evalute_models(RA_1_20_Datasets, [s for s in listing if 'ra_1_20' in s])
evalute_models(RA_2_20_Datasets, [s for s in listing if 'ra_2_20' in s])
evalute_models(RA_3_20_Datasets, [s for s in listing if 'ra_3_20' in s])
evalute_models(RA_2_5_Datasets, [s for s in listing if 'ra_2_5' in s])


In [19]:
with open('output.txt', 'w') as f:
     f.write(cap.stdout)

In [20]:
p = re.compile("Loss (.*) Accuracy (.*)\r")
result = p.findall(cap.stdout)

q = re.compile("/home/ec2-user/SageMaker/checkpoints/(.*)\r")
result2 = q.findall(cap.stdout)

r = re.compile("Test dataset: (.*)\n")
result3 = r.findall(cap.stdout)

acc = [e+(r2,r3) for e,r2,r3 in zip(result, result2, result3)]

In [21]:
df = pd.DataFrame(acc, columns =['Loss', 'Accuracy', 'Checkpoint', 'Testset'])
df.to_csv('/home/ec2-user/SageMaker/experiments/augmented_testsets/augmentedtestsets_results.csv')
df

,Loss,Accuracy,Checkpoint,Testset
0,1.6369,0.6328,resnet_basic_32_ra_1_20_checkpoint_00400.pth,CIFAR10_RA_1_20
1,1.9277,0.5220,resnet_basic_32_ra_1_20_checkpoint_00050.pth,CIFAR10_RA_1_20
2,5.1619,0.3440,resnet_basic_32_ra_1_20_checkpoint_00400.pth,CIFAR101_RA_1_20
3,3.1455,0.4125,resnet_basic_32_ra_1_20_checkpoint_00050.pth,CIFAR101_RA_1_20
4,3.9263,0.3291,wrn_28_10_ra_2_20_checkpoint_00050.pth,CIFAR10_RA_2_20
5,1.6370,0.4858,wrn_28_10_ra_2_20_checkpoint_00400.pth,CIFAR10_RA_2_20
6,1.5669,0.4909,resnext_29_4x64d_ra_2_20_checkpoint_00400.pth,CIFAR10_RA_2_20
7,4.8382,0.1921,resnext_29_4x64d_ra_2_20_checkpoint_00050.pth,CIFAR10_RA_2_20
8,3.4982,0.3270,wrn_28_10_ra_2_20_checkpoint_00050.pth,CIFAR101_RA_2_20
9,2.5823,0.3185,wrn_28_10_ra_2_20_checkpoint_00400.pth,CIFAR101_RA_2_20


In [22]:
# Move the results to the appropriate file in the GitHub Repo
if bucket == 'sagemaker-june29':
    !cp '/home/ec2-user/SageMaker/experiments/augmented_testsets/augmentedtestsets_results.csv' '/home/ec2-user/SageMaker/w210-capstone/analysis/supporting_data/augmentedtestset_results1.csv'
elif bucket == 'sagemaker-may29':
    !cp '/home/ec2-user/SageMaker/experiments/augmented_testsets/augmentedtestsets_results.csv' '/home/ec2-user/SageMaker/w210-capstone/analysis/supporting_data/augmentedtestset_results2.csv'
else:
    print("New bucket!  Please register with script.")  